# Define variables J, z, s,t

In [21]:
import numpy as np
import math
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler


z = np.arange(0.01,0.99,1/100)
N = len(z)
J = np.arange(0,100,2.0)
print(J)
print(np.kron(J,z).shape[0])

[ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18. 20. 22. 24. 26. 28. 30. 32. 34.
 36. 38. 40. 42. 44. 46. 48. 50. 52. 54. 56. 58. 60. 62. 64. 66. 68. 70.
 72. 74. 76. 78. 80. 82. 84. 86. 88. 90. 92. 94. 96. 98.]
4900


In [2]:
# generate random floating point values for s and t and F
# S and T
from random import seed
from random import random
import array as arr
#random number generator for s
seed(1)
s = arr.array('f')
t = arr.array('f')
    # generate random numbers between 0-1
for _ in range(100):
    #make s
    value_s = random()
    value_s = -1 + (value_s * (0 +1))
    s.append(value_s)
    
    #make t given restriction with s
    value_t = np.random.uniform(low=-value_s-1, high=0)
    t.append(value_t)
    #loop throygh s, and add t sch that it satisifes s+t < -1
s = np.array(s)
t = np.array(t)

#generate random momentum transfers t E (-1,0)
seed(1)
t_individual = arr.array('f')
# generate random numbers between 0-1
for _ in range(100):
    value = random()
    value = -1 + (value * (0 +1))
    t_individual.append(value)


# Define test set of variables J, z, s,t

In [3]:
# z = np.arange(0.01,0.99,1/1000)
# N = len(z)
# J = np.arange(0,1000,2.0)
# print(z,J)
# X = [[j, Z] for Z in z for j in J]

# X = np.array(X)
# print(X.shape)

# Define validation set of variables J, z, s,t

# # Define functions to be used in loss function

In [14]:
###Calculate M given array of z and J, for specific points (s,t)
import scipy
import scipy.integrate as integrate
from scipy.integrate import quad
import scipy.special


def Mst_Mts(s,t,z,J):
    return (M_withoutF(s,t,z,J) - M_withoutF(t,s,z,J))/N

def M_withoutF(s,t,z,J):   # use eq 2.6
    sum = 0
    A= []
    B = []
    for j in range(len(J)):
        for Z in z:
            a = 2/math.pi
            b = a*Z
            c = b*(1+Z*t)*(2+Z*t)
            d = c*np.array(P_J(1+2*Z*t, j))
            e = (1-Z*t)*(1+Z*(s+t))*s*(s+t)
            sum += d/e
            B.append(sum)
       # A.append(B)
    B= np.array(B)
    #A2 = np.array(A)
    return B

def P_J(x, J):
    #A =[]
    #for j in range(len(J)):
    B = scipy.special.lpmv(0, J, x)
    C = 16*math.pi*(2*J +1)
    D = C*B
    return D

def Gamma(z):
    return scipy.special.gamma(z)

def hypergeometric(a,b,c,d):
    return scipy.special.hyp2f1(a,b,c,d)

In [15]:
### K^(alpha)
###input = array of t_i, J_j and m_j
#Y sums over j, K sums over i.
def K_alpha(t, J, z):
    sum = 0
    k = []
    for j2 in J[0:]:
        for z2 in z[0:]:
            
            A = (-2*t/math.pi)
            B = A*z2
            C = B*(2 +t*z2)
            D = C*(P_J(1+2*z2*t, j2))
            sum = D
            k.append(sum)
            sum = 0
    
    return k

In [17]:
def K_Beta(J,z):
    sum =0
    A = []
    for j in J:
        for Z in z:
            sum += (4/math.pi)*Z**(3)*P_J(1,j)
            A.append(sum/N)
        
    return A

# Use functions above to make arrays to be used in loss function# 

In [18]:
#Make Mst - mts array with s and t as scalars
# Making s and t tuples to loop over Mst_Mts function 
#X = tf.stack([z, J], axis=1)

a = np.transpose(s)
b = np.transpose(t)
M=[]
#s0t0, s1t1, s2t2 ..
for i in range(len(s)):
    M.append(Mst_Mts(s[i],t[i],z,J))    

    
# for i in s[0:]:
#     for j in t[0:]:
#         print(i, j)
#         M.append(Mst_Mts(i,j,z,J))    
        
M= np.array(M)   
M =tf.convert_to_tensor(M, tf.float64)
print(np.max(M))

451466.9527584393


In [19]:
K_A = []
for t1 in t_individual[0:]:
    value = K_alpha(t1,J,z)
    #should be appending vectors
    K_A.append(value)
K_A= np.array(K_A)
K_A =tf.convert_to_tensor(K_A, tf.float64)
print(np.max(K_A))

868.3139844818685


In [20]:
K_B = K_Beta(J,z)
K_B= np.array(K_B)
K_B = np.transpose(K_B)
K_B =tf.convert_to_tensor(K_B, tf.float64)
print(np.max(K_B))



76071.44159999996


In [10]:
def Custom_Loss_Function(y_true, y_pred):
    
    y_pred = y_pred+0.1
           
    c  = 1 
    alpha = 10**6
    beta = 10**2

    # first term
    Mf = tf.tensordot(M, y_pred, axes=1)
    
    Mf = tf.square(Mf)
    sum1 = tf.reduce_sum(Mf)
    
    sum1 = sum1/451466.9527584393
   
    
    
    #second term
    Ka = tf.tensordot(K_A, y_pred, axes=1)
    sum2 = 0.0
    Ka = Ka - c
    Ka = tf.square(Ka)
    sum2 = tf.reduce_sum(Ka)
    sum2 = sum2 /868.3139844818685
    

    #third term
    sum3 = 0.0
    Kb = tf.tensordot(K_B, y_pred, axes=1)
    sum3 = Kb
    sum3 = sum3/76071.44159999996
    sum3 = sum3
    
    value = sum1 + alpha*sum2 -beta*sum3 

    return value

In [11]:
# # #testing Custom_Loss_Function
F = np.ones(4900,)
print(Custom_Loss_Function(F, F))

tf.Tensor(9.839157930888297e+17, shape=(), dtype=float64)
